# Introduction

In this notebook, we want to study the two datasets in `beerAdv_beer_brewery.tsv` and in `rateBeer_beer_brewery.tsv`. The columns of these datasets are like this:

            beer_id ¦ beer_name ¦ brewery_id ¦ brewery_name
            
The idea of this notebook is to create a similarity measure in order to find the same elements in both these datasets. Once we have this similarity measure, we want to find the limit X such that:
- All pairs with a similarity value of $X' > X$ are considered as the same beer
- All pairs with a similarity value of $X'' < X$ are not taken into account.

In order to achieve this, this notebook is composed of three parts. The first part is some sort of simple Data Analysis. Then there will be the creation of the pairs and the similarity value. Finally the part to find the best value $X$.

In [1]:
# Usefule imports
import pandas as pd
import numpy as np
import codecs

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Data Analysis

First, we need to analyze the data. It's a simple data analysis since we only have two usefull features: `beer_name` and `brewery_name`.

In [2]:
# Datasets
beerAdvocate_dataset = './data/beerAdv_beer_brewery.tsv'
rateBeer_dataset = './data/rateBeer_beer_brewery.tsv'

In [3]:
columns = ['beer_id', 'beer_name', 'brewery_id', 'brewery_name']

In [4]:
# Load the dataset with pandas
beerAdvocate = pd.read_table(beerAdvocate_dataset, header=None)
beerAdvocate.columns = columns
rateBeer = pd.read_table(rateBeer_dataset, header=None)
rateBeer.columns = columns

Print the two datasets

In [5]:
beerAdvocate.head()

,beer_id,beer_name,brewery_id,brewery_name
0,14348,Eisbrau Czech,1,"Plzensky Prazdroj, a. s."
1,19099,Primus,1,"Plzensky Prazdroj, a. s."
2,19123,Gambrinus Pale,1,"Plzensky Prazdroj, a. s."
3,19274,Urutislav,1,"Plzensky Prazdroj, a. s."
4,41294,Pilsner Urquell 3.5%,1,"Plzensky Prazdroj, a. s."


In [6]:
rateBeer.head()

,beer_id,beer_name,brewery_id,brewery_name
0,4,Abita Bock,1,Abita Brewing Company
1,10731,Abita Louisiana Red Ale,1,Abita Brewing Company
2,114065,Abita Select Pecan Brown Ale,1,Abita Brewing Company
3,114981,Abita Select Amber Ale,1,Abita Brewing Company
4,117017,Abita American Wheat,1,Abita Brewing Company


We know that the ids will not be the same in both datasets. Therefore, we can just drop the columns.

In [7]:
beerAdvocate = beerAdvocate.drop(['beer_id', 'brewery_id'], axis=1)
rateBeer = rateBeer.drop(['beer_id', 'brewery_id'], axis=1)

We just stop two minutes here. We can see a problem with the encoding of the file for *beerAdvocate*. 

In [8]:
name = beerAdvocate.ix[12].beer_name
print(name)

Master PolotmavÃ½ 13Â°


The encoding here is **latin_1**. Therefore, we can find the UTF-8 encoding by doing this:

In [9]:
new_name = bytes(name,'latin_1').decode('utf-8')
print(new_name)

Master Polotmavý 13°


Let's do it everywhere for *beerAdvocate*!

In [10]:
def decode_from_latin_1(string):
    try:
        return bytes(string ,'latin_1').decode('utf-8')
    except:
        return string

In [11]:
for i in range(len(beerAdvocate)):
    # Change the name of the beer
    beerAdvocate.set_value(i, 'beer_name', decode_from_latin_1(beerAdvocate.ix[i].beer_name))
    beerAdvocate.set_value(i, 'brewery_name', decode_from_latin_1(beerAdvocate.ix[i].brewery_name))


In [12]:
beerAdvocate.ix[12].beer_name

'Master Polotmavý 13°'

Let's print the number of entries in each dataset.

In [13]:
print("Number of rows in beerAdvocate: %i"%(len(beerAdvocate)))
print("Number of rows in rateBeer: %i"%(len(rateBeer)))

Number of rows in beerAdvocate: 66056
Number of rows in rateBeer: 110359


Let's check the number of **unique** beers to see if it matches the number of rows in the dataset.

In [14]:
unique_beers_beerAdvocate = beerAdvocate.beer_name.unique()
unique_beers_rateBeer = rateBeer.beer_name.unique()

print("Number of unique beers in beerAdvocate: %i"%(len(unique_beers_beerAdvocate)))
print("Number of unique beers in rateBeer: %i"%(len(unique_beers_rateBeer)))

Number of unique beers in beerAdvocate: 56855
Number of unique beers in rateBeer: 110302


It's already interesting to see that some beers have the same name in both datasets. Let's take one of them and see if they have the same `brewery_name`.

In [15]:
duplicated_beers_beerAdvocate = beerAdvocate[beerAdvocate.beer_name.duplicated()].beer_name.unique()
duplicated_beers_rateBeer = rateBeer[rateBeer.beer_name.duplicated()].beer_name.unique()

print("Number of duplicated beer names in beerAdvocate: %i"%(len(duplicated_beers_beerAdvocate)))
print("Number of duplicated beer names in rateBeer: %i"%(len(duplicated_beers_rateBeer)))

Number of duplicated beer names in beerAdvocate: 2707
Number of duplicated beer names in rateBeer: 55


In [16]:
dup_beer_dup_brewery = []

# Create list of tuples. 
#   First entry is the name of the duplicated beer
#   Second entry is the list of Brewery that is duplicated
#   Third entry is the list of indices to remove them easily.
for dup_beer in duplicated_beers_beerAdvocate:
    subdf = beerAdvocate[beerAdvocate.beer_name == dup_beer]
    if any(subdf.brewery_name.duplicated()):
        dup_beer_dup_brewery.append((dup_beer, list(subdf[subdf.brewery_name.duplicated()]["brewery_name"]), list(subdf[subdf.brewery_name.duplicated()].index)))

In [17]:
print("Number of beers with duplicated brewery in beerAdvocate: %i"%(len(dup_beer_dup_brewery)))

Number of beers with duplicated brewery in beerAdvocate: 286


Just check how the function `duplicated` works. For example, if we have two times the same brewery, this means that we would have 3 times the same beer and the same brewery in the data set.

In [18]:
dup_beer_dup_brewery[3]

('Saison',
 ['Triumph Brewing Company', 'Triumph Brewing Company'],
 [22563, 43875])

In [19]:
beerAdvocate[(beerAdvocate.beer_name == "Saison") & (beerAdvocate.brewery_name == "Triumph Brewing Company")]

,beer_name,brewery_name
17881,Saison,Triumph Brewing Company
22563,Saison,Triumph Brewing Company
43875,Saison,Triumph Brewing Company


Verfied. Therefore, we can just remove the indices.

In [20]:
indices_to_remove = []
for i in range(len(dup_beer_dup_brewery)):
    indices_to_remove.extend(dup_beer_dup_brewery[i][2])

In [21]:
# Remove the indices in the dataset
beerAdvocate = beerAdvocate.drop(indices_to_remove, axis=0)
beerAdvocate.index = range(len(beerAdvocate))
beerAdvocate.tail()

,beer_name,brewery_name
65668,Foster's Gold,Foster's Group Limited
65669,Foster's Special Bitter,Foster's Group Limited
65670,Foster's Lager,Foster's Group Limited
65671,Hefe Weissbier,Bayerische Löwenbrauerei
65672,Hefeweissbier Dunkel,Bayerische Löwenbrauerei


Now, we need to the same for the rateBeer dataset.

In [22]:
dup_beer_dup_brewery = []

# Create list of tuples. 
#   First entry is the name of the duplicated beer
#   Second entry is the list of Brewery that is duplicated
#   Third entry is the list of indices to remove them easily.
for dup_beer in duplicated_beers_rateBeer:
    subdf = rateBeer[rateBeer.beer_name == dup_beer]
    if any(subdf.brewery_name.duplicated()):
        dup_beer_dup_brewery.append((dup_beer, list(subdf[subdf.brewery_name.duplicated()]["brewery_name"]), list(subdf[subdf.brewery_name.duplicated()].index)))

In [23]:
print("Number of beers with duplicated brewery in beerRate: %i"%(len(dup_beer_dup_brewery)))

Number of beers with duplicated brewery in beerRate: 4


In [24]:
dup_beer_dup_brewery

[('Traugott Simon Export', ['Udo Täubrich Betreuungs'], [9819]),
 ('Big Horn Saison',
  ['Big Horn Brewing Company (Ram International)'],
  [19577]),
 ('Yukon Lead Dog Ale', ['Yukon Brewing Company'], [37365]),
 ('Prison Brews Winter Ale', ['Prison Brews'], [77632])]

In [25]:
indices_to_remove = []
for i in range(len(dup_beer_dup_brewery)):
    indices_to_remove.extend(dup_beer_dup_brewery[i][2])

In [26]:
# Remove the indices in the dataset
rateBeer = rateBeer.drop(indices_to_remove, axis=0)
rateBeer.index = range(len(rateBeer))
rateBeer.tail()

,beer_name,brewery_name
110350,Jelling Bryghus Frode Fredegod,Jelling Bryghus
110351,Jelling Bryghus Poppo,Jelling Bryghus
110352,Jelling Bryghus Jelling Jól,Jelling Bryghus
110353,Jelling Bryghus Jalunki,Jelling Bryghus
110354,Jelling Bryghus Sildeglimt,Jelling Bryghus


### Save the cleaned data set

Now that the datasets are cleaned, we can save them in a CSV format.

In [27]:
beerAdvocate.to_csv('./data/beerAdvocate_cleaned.csv', index=False)
rateBeer.to_csv('./data/rateBeer_cleaned.csv', index=False)

# Similarity

** YOU CAN DIRECTLY START FROM HERE. NO NEED TO REDO THE CLEANING OF THE DATASETS! **

Now, we want to match beers between the two different datasets. TO BE CONTINUED...

In [31]:
beerAdvocate = pd.read_csv('./data/beerAdvocate_cleaned.csv', encoding='latin_1')
rateBeer = pd.read_csv('./data/rateBeer_cleaned.csv', encoding='latin_1')

In [33]:
beerAdvocate.head(15)

,beer_name,brewery_name
0,Eisbrau Czech,"Plzensky Prazdroj, a. s."
1,Primus,"Plzensky Prazdroj, a. s."
2,Gambrinus Pale,"Plzensky Prazdroj, a. s."
3,Urutislav,"Plzensky Prazdroj, a. s."
4,Pilsner Urquell 3.5%,"Plzensky Prazdroj, a. s."
5,Pilsner Urquell,"Plzensky Prazdroj, a. s."
6,Gambrinus Excelent (Gambrinus 11),"Plzensky Prazdroj, a. s."
7,Pilsner Urquell Kvasnicový (Unfiltered Yeast B...,"Plzensky Prazdroj, a. s."
8,Gambrinus Svetly,"Plzensky Prazdroj, a. s."
9,Gambrinus,"Plzensky Prazdroj, a. s."


In [34]:
rateBeer.head(15)

,beer_name,brewery_name
0,Abita Bock,Abita Brewing Company
1,Abita Louisiana Red Ale,Abita Brewing Company
2,Abita Select Pecan Brown Ale,Abita Brewing Company
3,Abita Select Amber Ale,Abita Brewing Company
4,Abita American Wheat,Abita Brewing Company
5,Abita Select Imperial Stout,Abita Brewing Company
6,Abita Select Double IPA,Abita Brewing Company
7,Abita Select Weizenbock,Abita Brewing Company
8,Abita Select Imperial Kolsch,Abita Brewing Company
9,Abita SOS (Save Our Shore),Abita Brewing Company
